Загружаем данные:

In [1]:
from sklearn import datasets
import numpy as np

iris = datasets.load_iris()
X = iris.data  # we take all features.
#X = iris.data[:, :2]  # we only take the first two features.
Y = iris.target

Преобразуем ответы (отделять будем setosa (метка класса 0) от остального):

In [2]:
for i in xrange(len(Y)):
    if Y[i] == 0:
        Y[i] = 1
    else:
        Y[i] = -1

Разбиваем данные на две части: одна для обучения, другая для тестирования:

In [12]:
from sklearn.cross_validation import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.15)

Функции для вычисления loss, которую и будем оптимизировать, (ее можно менять, я пробовал несколько вариантов, но для примера оставил сигмоиду) и ее производной, а также непосредственно классификатор:

In [13]:
def L(t):
    #return np.e**(-t)
    #return (1-t)**2
    return 2/(1+np.e**t)

def L_derivative(t):
    #return -np.e**(-t)
    #return 2*(t-1)
    return -2*np.e**t/((1+np.e**t)**2)
    
class StochGrad:
    def __init__(self, learning_rate, smooth_parameter):
        self.learning_rate = learning_rate
        self.smooth_parameter = smooth_parameter
        
    w = np.zeros([1,1])
    
    def fit(self, X, Y):
        
        n = len(X)
        d = len(X[0])
        X = np.hstack([X, -np.ones([n,1])])
        
        self.w = np.random.uniform(-0.5/d, 0.5/d, d+1)
        Q = 0
        for i in xrange(n):
            Q += L(np.dot(self.w, X[i])*Y[i])
        convergence = False
        
        iterations = 0
        while (not convergence):
            print self.w
            print Q
            w_last = self.w
            Q_last = Q
            
            i = np.random.randint(n)
            eps = L(np.dot(self.w, X[i])*Y[i])
            self.w = self.w - self.learning_rate*L_derivative(np.dot(self.w, X[i])*Y[i])*Y[i]*X[i]
            print L_derivative(np.dot(self.w, X[i])*Y[i])
            Q = (1-self.smooth_parameter)*Q + self.smooth_parameter*eps
            
            if (abs((Q-Q_last)/Q) < 0.01):
                convergence = True
            iterations += 1
            
    def predict(self, X):
        
        n = len(X)
        X = np.hstack([X, -np.ones([n,1])])
        Y = np.zeros(n)
        
        for i in xrange(n):
            Y[i] = np.sign(np.dot(self.w, X[i]))
            
        return Y

In [14]:
clf = StochGrad(0.1, 1.0/150)
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
Y_pred == Y_test 

[ 0.11398209 -0.04714994  0.09467651  0.05046343 -0.00027587]
159.551257499
-0.232412424169


array([ True, False,  True, False, False, False,  True, False, False,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False,  True,  True,  True], dtype=bool)

In [15]:
from sklearn import metrics

metrics.zero_one_loss(Y_test,Y_pred)

0.39130434782608692

Как-то странно и бредово работает